In [1]:
# Check to see if we're using a GPU
!nvidia-smi

Fri Oct 13 22:05:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 101 food classes: working with less data
Our goal is to beath the original Food101 paper with 10% of the training data.

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2023-10-13 22:07:05--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.207, 74.125.201.207, 74.125.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  80.5MB/s    in 23s     

2023-10-13 22:07:29 (67.4 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [3]:
import zipfile
zip_ref = zipfile.ZipFile("101_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [7]:
# How many images in each folder?
import os

# Walk through -1 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("101_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in `{dirpath}`")

There are 2 directories and 0 images in `101_food_classes_10_percent`
There are 101 directories and 0 images in `101_food_classes_10_percent/train`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/french_toast`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/frozen_yogurt`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/baklava`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/donuts`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/french_onion_soup`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/bruschetta`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/ramen`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/cheesecake`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/hot_dog`
There are 0 directories and 75 images in `101_food_classes_10_p

In [8]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [9]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)  # don't shuffle test data for prediction analysis

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## Train a big dog model with transfer learning on 10% of 101 food classes

In [10]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [11]:
# Create data augmentation layer to incorporate it righ into the model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    # preprocessing.Rescaling(1/255.)   # rescale inputs of images to between 0 and 1, required for models like ResNet50
])

In [12]:
# Setup the base model and freeze its layer (this will extract features)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Set up model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs) # augment images (only happens during training phase)
x = base_model(x, training=False) # put the base model in inference mode so weights which need to stay frozen, stay frozen
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

16705208/16705208 [==============================] - 0s 0us/step


In [13]:
# Get a summary of model we've created
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, None, None, 3)     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_avg_pool_layer (Glo  (None, 1280)              0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4178952 (15.94 MB)
Trainable params: 129381 (50

In [16]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [17]:
# Fit the model
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=10,
                                           steps_per_epoch=len(train_data_all_10_percent),
                                           validation_data=test_data,
                                           validation_steps=int(0.15 * len(test_data)),
                                           # Track model training logs
                                           callbacks=[checkpoint_callback])

Epoch 1/10
237/237 [==============================] - 126s 454ms/step - loss: 3.3897 - accuracy: 0.2752 - val_loss: 2.5711 - val_accuracy: 0.4399
Epoch 2/10
237/237 [==============================] - 80s 336ms/step - loss: 2.2059 - accuracy: 0.4931 - val_loss: 2.0827 - val_accuracy: 0.4992
Epoch 3/10
237/237 [==============================] - 71s 295ms/step - loss: 1.8228 - accuracy: 0.5690 - val_loss: 1.9360 - val_accuracy: 0.5185
Epoch 4/10
237/237 [==============================] - 65s 274ms/step - loss: 1.6057 - accuracy: 0.6050 - val_loss: 1.8533 - val_accuracy: 0.5294
Epoch 5/10
237/237 [==============================] - 60s 252ms/step - loss: 1.4506 - accuracy: 0.6400 - val_loss: 1.8087 - val_accuracy: 0.5360
Epoch 6/10
237/237 [==============================] - 59s 245ms/step - loss: 1.3291 - accuracy: 0.6735 - val_loss: 1.7356 - val_accuracy: 0.5442
Epoch 7/10
237/237 [==============================] - 55s 229ms/step - loss: 1.2404 - accuracy: 0.6962 - val_loss: 1.7432 - val_a